In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from random import shuffle


In [2]:
inputSources = [
   "monthly_sp500_pca.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv",
   "monthly_gold_prices.csv"
]

In [3]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 2048,
    'accuracyThreshold': 0.5,
    'controlThreshold': 0.7
}


In [4]:
class ReflexiveMemory:
  def __init__(self, reflexSize, dimensions_dense, dimensions_sparse_sp):
    self.acKey0 = None
    self.pairs = {}
    self.tableSize  = reflexSize
    self.dimensions_dense = dimensions_dense
    self.dimensions_sparse_sp = dimensions_sparse_sp

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data
        
      table_entries = 0
      oldKey1 = None
      oldKey2 = None
      oldTime = datetime.now()
      for key1, value1 in self.pairs.items():
        table_entries = table_entries + len(value1.items())
        for key2, value2 in value1.items():
          if value2['time'] < oldTime:
            oldKey1 = key1
            oldKey2 = key2
            oldTime = value2['time']
      if table_entries > self.tableSize:
        del self.pairs[oldKey1][oldKey2]
        if len(self.pairs[oldKey1].items()) == 0:
          del self.pairs[oldKey1]

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( self.dimensions_dense )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr


In [5]:
class ControlUnit:
  def __init__(self, controlThreshold):
    self.anomalyRM = []
    self.anomalyTM = []
    self.anomalyNU = []
    self.anomalyCU = []
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []
    self.historyCU = []
    self.controlThreshold = controlThreshold

  def anomalyScore(self, y, x):
      if np.count_nonzero(y) != 0:
          return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
      return 1

  def compute(self, denseColumns1, sp, tm, rm):

    if rm.acKey0 is not None:

      denseColumns0 = SDR( rm.dimensions_dense )
      denseColumns0.sparse = list(map(int, rm.acKey0.split('-')))

      tm.activateDendrites(True)
      predictiveCells = tm.getPredictiveCells()

      predictiveColumns = SDR( rm.dimensions_sparse_sp )
      predictiveColumns.sparse = list(set(sorted(list(np.where(predictiveCells.dense == 1)[0]))))

      reflexiveColumns = SDR( rm.dimensions_sparse_sp )
      reflexiveCount, denseReflexiveColumns = rm.predict(denseColumns0)
      if denseReflexiveColumns is not None:
        sp.compute(denseReflexiveColumns, False, reflexiveColumns)

      activeColumns0 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns0, False, activeColumns0)

      activeColumns1 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns1, False, activeColumns1)

      self.historyRM.append( reflexiveColumns.dense )
      self.historyTM.append( predictiveColumns.dense )
      self.historyGT.append( activeColumns1.dense )

      self.anomalyNU.append(tm.anomaly)
      self.anomalyRM.append( self.anomalyScore(activeColumns1.dense, reflexiveColumns.dense) )
      self.anomalyTM.append( self.anomalyScore(activeColumns1.dense, predictiveColumns.dense) )

      #  = sum(self.anomalyRM[-8:])/len(self.anomalyRM[-8:])
      if self.anomalyRM[-1] < self.anomalyTM[-1]:
        self.anomalyCU.append( self.anomalyRM[-1] )
        self.historyCU.append( self.historyRM[-1] )
      else:
        self.anomalyCU.append( self.anomalyTM[-1] )
        self.historyCU.append( self.historyTM[-1] )

#      if overlapRM < (1 - 0.5):
#        oldKey1 = '-'.join(map(str, denseColumns0.sparse))
#        oldKey2 = '-'.join(map(str, denseReflexiveColumns.sparse))
#        if reflexiveCount > 1:
#          self.pairs[oldKey1][oldKey2]["count"] = reflexiveCount - 1
#        else:
#          del self.pairs[oldKey1][oldKey2]
#          if len(self.pairs[oldKey1].items()) == 0:
#            del self.pairs[oldKey1]
  

In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    tm_infer_tm = 0
    tm_infer_rm = 0

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] , config['sp']['inputDimensions'], config['sp']['columnDimensions'])
    cu = ControlUnit( config['controlThreshold'] )

    try:
        
        for count, record in enumerate(records):
            
            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            consumption = float(record[1])
            consumptionBits = scalarEncoder.encode(consumption)
            encoding = SDR( consumptionBits )

            cu.compute(encoding, sp, tm, rm)

            rm.add(encoding)

            activeColumns = SDR( sp.getColumnDimensions() )
            sp.compute(encoding, learn_sp, activeColumns)

            tmp_tm = time.time()
            tm.compute(activeColumns, learn=learn_tm)
            tm_infer_tm = tm_infer_tm + (time.time() - tmp_tm)

            tmp_tm = time.time()
            rm.predict(encoding)
            tm_infer_rm = tm_infer_rm + (time.time() - tmp_tm)

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    def roc_auc_score_multiclass(y_true, y_pred):
        scores = []
        for y_class in set(y_true):
            y_true_class = [True if x == y_class else False for x in y_true]
            y_pred_class = [True if x == y_class else False for x in y_pred]
            scores.append(roc_auc_score(y_true_class, y_pred_class))
        return sum(scores) / len(scores)

    def match(y, x, idx1, cu, accuracyThreshold):
        n_samples = len(y)
        score1 = cu.anomalyScore(y[idx1], x[idx1])
        if score1 > (1 - accuracyThreshold):
            idx_closest = None
            score_closest = None
            for idx2 in range(n_samples):
                score2 = cu.anomalyScore(y[idx2], x[idx1])
                if score_closest is None or score_closest > score2:
                    score_closest = score2
                    idx_closest = idx2
            return idx_closest
        return idx1

    def calculateMetricsAnomaly(anomaly_scores, config, suffix):

        metric = {}

        anomaly_probability = []
        anomaly_history = AnomalyLikelihood(config["anomaly"]["period"])
        for anomaly_value in anomaly_scores:
            anomaly_probability.append( anomaly_history.compute(anomaly_value) )

        metric['anomaly-avg-'+suffix] = sum(anomaly_scores) / len(anomaly_scores)
        metric['anomaly-samples-'+suffix] = len(anomaly_scores)
        metric['anomaly-prob-avg-'+suffix] = np.count_nonzero(anomaly_probability) / len(anomaly_probability)

        return metric

    def calculateMetrics(Y_dataset, X_dataset, anomaly_scores, total_infe_time, config, cu, suffix):

        metric = {}

        n_samples = len(Y_dataset)

        Y_labels = list(range(n_samples))
        X_labels = [ match(Y_dataset, X_dataset, idx, cu, config['accuracyThreshold']) for idx in range(n_samples)]
        precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
        metric['total-infer-time-'+suffix] = total_infe_time
        metric['infer-time-'+suffix] = total_infe_time / len(anomaly_scores)
        metric['accuracy-'+suffix] = accuracy_score(Y_labels, X_labels)
        metric['precision-'+suffix] = precision
        metric['recall-'+suffix] = recall
        metric['fscore-'+suffix] = fscore
        metric['support-'+suffix] = support
        metric['auc-'+suffix] = roc_auc_score_multiclass(Y_labels, X_labels)

        metric.update( calculateMetricsAnomaly(anomaly_scores, config, suffix) )

        return metric
    
    tm_infer_cu = 0
    avg_infer_time_rm = tm_infer_rm / len(cu.anomalyRM)
    avg_infer_time_tm = tm_infer_tm / len(cu.anomalyTM)
    for idx, eval_anomaly in enumerate(cu.anomalyCU):
        if eval_anomaly == cu.anomalyRM[idx]:
            tm_infer_cu = tm_infer_cu + avg_infer_time_rm
        else:
            tm_infer_cu = tm_infer_cu + avg_infer_time_tm

    metric = {}
    metric['dataset'] = dataset
    metric.update( calculateMetrics(cu.historyGT, cu.historyRM, cu.anomalyRM, tm_infer_rm, config, cu, 'rm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyTM, cu.anomalyTM, tm_infer_tm, config, cu, 'tm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyCU, cu.anomalyCU, tm_infer_cu, config, cu, 'cu') )
    metric.update( calculateMetricsAnomaly(cu.anomalyNU, config, 'nupic') )

    dataset_metrics.append(metric)
    pbar.update(1)
    # break

pbar.close()

100%|██████████| 15/15 [05:37<00:00, 22.51s/it]


In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)

In [8]:
table_1_features = ['dataset','accuracy-cu','accuracy-rm','accuracy-tm','anomaly-avg-cu','anomaly-avg-rm','anomaly-avg-tm','anomaly-avg-nupic']
df[table_1_features]

,dataset,accuracy-cu,accuracy-rm,accuracy-tm,anomaly-avg-cu,anomaly-avg-rm,anomaly-avg-tm,anomaly-avg-nupic
0,monthly_sp500_pca.csv,0.434491,0.177940,0.414991,0.565373,0.803186,0.580777,0.580777
1,weekly_dow_jones.csv,0.035594,0.009620,0.033189,0.934470,0.984608,0.935994,0.935994
2,weekly_nasdaq.csv,0.222115,0.075000,0.217308,0.752838,0.906864,0.757732,0.757786
3,weekly_sp500.csv,0.353196,0.131187,0.347429,0.617752,0.845026,0.625941,0.625968
4,monthly_vix_close.csv,0.975556,0.690123,0.971111,0.041790,0.326005,0.047743,0.047990
5,monthly_vix_high.csv,0.956049,0.679259,0.944691,0.062023,0.349348,0.075261,0.075504
6,monthly_vix_low.csv,0.970370,0.772840,0.960247,0.047226,0.308292,0.058693,0.058903
7,monthly_vix_open.csv,0.963951,0.731852,0.953333,0.055364,0.346145,0.066948,0.067123
8,daily_natural_gas.csv,0.994998,0.955847,0.994136,0.005907,0.065410,0.009682,0.009855
9,daily_oil_prices.csv,0.994699,0.904096,0.993494,0.018765,0.221058,0.021561,0.021677


In [9]:
table_2_features = ['dataset','anomaly-avg-cu','anomaly-avg-rm','anomaly-avg-tm','anomaly-avg-nupic','anomaly-prob-avg-cu','anomaly-prob-avg-rm','anomaly-prob-avg-tm','anomaly-prob-avg-nupic','anomaly-samples-cu','anomaly-samples-rm','anomaly-samples-tm','anomaly-samples-nupic']
df[table_2_features]

,dataset,anomaly-avg-cu,anomaly-avg-rm,anomaly-avg-tm,anomaly-avg-nupic,anomaly-prob-avg-cu,anomaly-prob-avg-rm,anomaly-prob-avg-tm,anomaly-prob-avg-nupic,anomaly-samples-cu,anomaly-samples-rm,anomaly-samples-tm,anomaly-samples-nupic
0,monthly_sp500_pca.csv,0.565373,0.803186,0.580777,0.580777,0.391225,0.391225,0.391225,0.391225,1641,1641,1641,1641
1,weekly_dow_jones.csv,0.934470,0.984608,0.935994,0.935994,0.519481,0.519481,0.519481,0.519481,2079,2079,2079,2079
2,weekly_nasdaq.csv,0.752838,0.906864,0.757732,0.757786,0.519712,0.519712,0.519712,0.519712,2080,2080,2080,2080
3,weekly_sp500.csv,0.617752,0.845026,0.625941,0.625968,0.519942,0.519942,0.519942,0.519942,2081,2081,2081,2081
4,monthly_vix_close.csv,0.041790,0.326005,0.047743,0.047990,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
5,monthly_vix_high.csv,0.062023,0.349348,0.075261,0.075504,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
6,monthly_vix_low.csv,0.047226,0.308292,0.058693,0.058903,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
7,monthly_vix_open.csv,0.055364,0.346145,0.066948,0.067123,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
8,daily_natural_gas.csv,0.005907,0.065410,0.009682,0.009855,0.827699,0.827699,0.827699,0.827699,5798,5798,5798,5798
9,daily_oil_prices.csv,0.018765,0.221058,0.021561,0.021677,0.879639,0.879639,0.879639,0.879639,8300,8300,8300,8300


In [10]:
table_3_features = ['dataset','infer-time-cu','infer-time-rm','infer-time-tm','total-infer-time-cu','total-infer-time-tm','total-infer-time-rm']
df[table_3_features]

,dataset,infer-time-cu,infer-time-rm,infer-time-tm,total-infer-time-cu,total-infer-time-tm,total-infer-time-rm
0,monthly_sp500_pca.csv,0.000242,0.000011,0.000337,0.397348,0.552632,0.018417
1,weekly_dow_jones.csv,0.000134,0.000009,0.000338,0.278262,0.701888,0.019160
2,weekly_nasdaq.csv,0.000231,0.000010,0.000354,0.479970,0.736157,0.020897
3,weekly_sp500.csv,0.000245,0.000010,0.000320,0.510137,0.665149,0.021279
4,monthly_vix_close.csv,0.000171,0.000011,0.000264,0.691713,1.069025,0.044819
5,monthly_vix_high.csv,0.000215,0.000012,0.000333,0.870725,1.348159,0.047818
6,monthly_vix_low.csv,0.000135,0.000010,0.000224,0.545159,0.907011,0.042409
7,monthly_vix_open.csv,0.000196,0.000011,0.000305,0.791896,1.233568,0.045805
8,daily_natural_gas.csv,0.000013,0.000007,0.000050,0.076688,0.287111,0.043055
9,daily_oil_prices.csv,0.000063,0.000009,0.000107,0.519689,0.887026,0.072683


In [11]:
table_4_features = ['dataset'] + sorted(list(set(df.columns) - (set(table_1_features) | set(table_2_features) | set(table_3_features))))
df[table_4_features]

,dataset,auc-cu,auc-rm,auc-tm,fscore-cu,fscore-rm,fscore-tm,precision-cu,precision-rm,precision-tm,recall-cu,recall-rm,recall-tm,support-cu,support-rm,support-tm
0,monthly_sp500_pca.csv,0.717073,0.588720,0.707317,0.419119,0.171347,0.399111,0.412657,0.169797,0.392548,0.434491,0.177940,0.414991,None,None,None
1,weekly_dow_jones.csv,0.517565,0.504572,0.516362,0.031667,0.008659,0.029903,0.030063,0.008498,0.028620,0.035594,0.009620,0.033189,None,None,None
2,weekly_nasdaq.csv,0.610871,0.537278,0.608466,0.211941,0.071245,0.206412,0.207733,0.070012,0.201900,0.222115,0.075000,0.217308,None,None,None
3,weekly_sp500.csv,0.676442,0.565385,0.673558,0.337761,0.127135,0.332122,0.331726,0.125701,0.326079,0.353196,0.131187,0.347429,None,None,None
4,monthly_vix_close.csv,0.987775,0.845023,0.985552,0.972540,0.686578,0.967951,0.971396,0.685954,0.966705,0.975556,0.690123,0.971111,None,None,None
5,monthly_vix_high.csv,0.978019,0.839590,0.972339,0.953331,0.676135,0.941571,0.952365,0.675605,0.940507,0.956049,0.679259,0.944691,None,None,None
6,monthly_vix_low.csv,0.985182,0.886392,0.980119,0.967642,0.769844,0.957119,0.966584,0.769212,0.956060,0.970370,0.772840,0.960247,None,None,None
7,monthly_vix_open.csv,0.981971,0.865893,0.976661,0.960565,0.727774,0.950164,0.959422,0.727110,0.949000,0.963951,0.731852,0.953333,None,None,None
8,daily_natural_gas.csv,0.997499,0.977920,0.997067,0.994521,0.954911,0.993571,0.994323,0.954727,0.993353,0.994998,0.955847,0.994136,None,None,None
9,daily_oil_prices.csv,0.997349,0.952042,0.996747,0.993629,0.901053,0.992273,0.993153,0.900293,0.991764,0.994699,0.904096,0.993494,None,None,None


In [12]:
df['cu-infer-speedup'] = 1 - (df['total-infer-time-cu'] / df['total-infer-time-tm'])
df['cu-accuracy-improvement'] = df['accuracy-cu'] - df['accuracy-tm']
table_5_features = ['dataset','cu-accuracy-improvement','cu-infer-speedup','accuracy-cu','accuracy-rm','accuracy-tm','total-infer-time-cu','total-infer-time-tm','total-infer-time-rm']
df[table_5_features]

,dataset,cu-accuracy-improvement,cu-infer-speedup,accuracy-cu,accuracy-rm,accuracy-tm,total-infer-time-cu,total-infer-time-tm,total-infer-time-rm
0,monthly_sp500_pca.csv,0.019500,0.280990,0.434491,0.177940,0.414991,0.397348,0.552632,0.018417
1,weekly_dow_jones.csv,0.002405,0.603553,0.035594,0.009620,0.033189,0.278262,0.701888,0.019160
2,weekly_nasdaq.csv,0.004808,0.348006,0.222115,0.075000,0.217308,0.479970,0.736157,0.020897
3,weekly_sp500.csv,0.005766,0.233048,0.353196,0.131187,0.347429,0.510137,0.665149,0.021279
4,monthly_vix_close.csv,0.004444,0.352950,0.975556,0.690123,0.971111,0.691713,1.069025,0.044819
5,monthly_vix_high.csv,0.011358,0.354138,0.956049,0.679259,0.944691,0.870725,1.348159,0.047818
6,monthly_vix_low.csv,0.010123,0.398950,0.970370,0.772840,0.960247,0.545159,0.907011,0.042409
7,monthly_vix_open.csv,0.010617,0.358044,0.963951,0.731852,0.953333,0.791896,1.233568,0.045805
8,daily_natural_gas.csv,0.000862,0.732899,0.994998,0.955847,0.994136,0.076688,0.287111,0.043055
9,daily_oil_prices.csv,0.001205,0.414122,0.994699,0.904096,0.993494,0.519689,0.887026,0.072683
